In [ ]:
import pandas as pd
import re
from pathlib import Path

value = 'mhs50'

def parse_limit_logs(value: str):
    # Use pathlib for modern, object-oriented path handling
    log_folder = Path('./Input_for_Limits_Plots')

    # Use glob to find all matching files directly, which is more efficient
    # than listing the whole directory and filtering manually.
    log_pattern = f'darkhiggs_*{value}*_obsAsymptoticLimits*'
    files = list(log_folder.glob(log_pattern))

    # Compile regular expressions once for efficiency
    filename_regex = re.compile(r'Mz(\d+\.?\d*)_mhs([\w\d.-]+)_Mdm(\d+\.?\d*)')

    # A mapping from text in the file to the desired column name
    # This avoids a long series of 'if' statements.
    limit_parsers = {
        'Observed Limit:': 'obs',
        'Expected 50.0%:': 'exp',
        'Expected 84.0%:': 'p1s',
        'Expected 16.0%:': 'm1s',
        'Expected 97.5%:': 'p2s',
        'Expected  2.5%:': 'm2s',
    }

    # Collect data in a list of dictionaries, a standard way to build a DataFrame
    all_results = []

    for ifile in files:
        filename = ifile.name

        # Use regex to robustly extract parameters from the filename
        match = filename_regex.search(filename)

        # Start a dictionary to hold data for this specific file
        # This is much cleaner than managing many separate lists.
        point_data = {
            'Mz': float(match.group(1)),
            'mhs': match.group(2), # mhs seems to be treated as a string/category
            'Mdm': float(match.group(3)),
        }

        found_limits = {}

        with ifile.open('r') as f:
            for line in f.readlines():
                for key, col_name in limit_parsers.items():
                    if key in line:
                        found_limits[col_name] = float(line.split('<')[1].strip())
                        break

        if len(found_limits) == len(limit_parsers):
            point_data.update(found_limits)
            all_results.append(point_data)

    # Create the DataFrame once from the list of dictionaries
    df = pd.DataFrame(all_results)
    # Sort by multiple columns in a single, efficient operation
    # reset_index is good practice after sorting to get a clean 0-based index.
    df = df.sort_values(by=['Mz', 'Mdm']).reset_index(drop=True)

    return df

In [ ]:
mhs50_df = parse_limit_logs('mhs50')
mhs70_df = parse_limit_logs('mhs70')
mhs90_df = parse_limit_logs('mhs90')
mhs110_df = parse_limit_logs('mhs110')
mhs130_df = parse_limit_logs('mhs130')
mhs150_df = parse_limit_logs('mhs150')

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as mticker
from matplotlib.lines import Line2D
import mplhep as hep
hep.style.use('CMS')

sys.path.append('../')

# --- Using your specific library import ---
from libs.limitlib import interpolate_rbf

cmap = mcolors.LinearSegmentedColormap.from_list("n", list(reversed([
    # '#fff5f0',
    '#fee0d2',
    '#ffffff',
    '#fcbba1',
    '#fc9272',
    '#fb6a4a',
    '#ef3b2c',
    '#cb181d',
    '#a50f15',
    '#67000d',
    # '#000000',
        ])))

def plot2d(df):
    fig, ax = plt.subplots(figsize=(14,10))
    hep.cms.label(data=True, year='2016-2018', lumi=138)#, loc=1)#, label="Preliminary")

    x = df['Mz']
    y = df['Mdm']
    obs = df['obs']

    contours_filled = np.log10(np.logspace(-1,1,7))
    contours_line = [0]

    def get_x_y_z(x,y,z):
        ix, iy, iz = interpolate_rbf(x,y,z,maxval=5000)
        iz [iy>ix] = 1e9 #* np.exp(-(iy/ix))
        if True:
            iz = np.log10(iz)
            iz[iz<min(contours_filled)] = min(contours_filled)
        return ix, iy, iz

    ix, iy, iz = get_x_y_z(x, y, obs)
    cf = ax.contourf(ix, iy, iz, levels=contours_filled, cmap=cmap)
    cb = fig.colorbar(cf, ax=ax)
    cb.set_label(r"95% CL observed upper limit on $\log_{10}(\mu)$", loc='center')
    for c in cf.collections:
        c.set_edgecolor("face")

    contour_lines_to_plot = [
        {'z_data': df['obs'], 'linestyle': [(0, (5, 0))], 'linewidth': 3, 'label': 'Observed'},
        {'z_data': df['exp'], 'linestyle': [(0, (5, 1))], 'linewidth': 3, 'label': 'Median expected'},
        {'z_data': df['p1s'], 'linestyle': [(0, (3, 3))], 'linewidth': 2, 'label': r'$\pm 1\sigma$ expected'},
        {'z_data': df['m1s'], 'linestyle': [(0, (3, 3))], 'linewidth': 2, 'label': None},
        {'z_data': df['p2s'], 'linestyle': [(0, (1, 5))], 'linewidth': 2, 'label': r'$\pm 2\sigma$ expected'},
        {'z_data': df['m2s'], 'linestyle': [(0, (1, 5))], 'linewidth': 2, 'label': None},
    ]

    for item in contour_lines_to_plot:
        cs = ax.contour(
            *get_x_y_z(x,y,item['z_data']),
            levels=contours_line,
            colors='black',
            linestyles=item['linestyle'],
            linewidths=item['linewidth']
        )
        if item.get('label'):
            cs.collections[0].set_label(item['label'])

    # --- Aesthetics and Labels ---
    mhs_val = int(df['mhs'].iloc[0])
    ax.set_ylim(mhs_val, 1500)
    ax.set_xlim(mhs_val, 5000)

    # 1. Draw the main label but WITHOUT the prime symbol.
    ax.set_xlabel(r"$m_{\mathrm{Z}}$ (TeV)", loc='center', fontsize=31)

    # 2. Manually add the prime symbol exactly where we want it.
    # This gives us full control over size, position, and thickness.
    #
    # YOU WILL LIKELY NEED TO FINE-TUNE THE X and Y COORDINATES BELOW.
    #
    ax.text(0.465, -0.09, r"$\prime$",              # The prime character itself
            transform=ax.transAxes,         # IMPORTANT: Use axes-relative coordinates
            fontsize=37,                    # A large font size for the prime
            fontweight='bold',              # Make it thick, as in your image
            ha='left',                      # Horizontal alignment
            va='center')                    # Vertical alignment

    ax.set_ylabel(r"$m_{\chi}$ (GeV)", loc='center', fontsize=31)

    def gev_to_tev(x, pos):
        return f'{x / 1000:g}'

    # Apply the custom formatter to the x-axis
    ax.xaxis.set_major_formatter(mticker.FuncFormatter(gev_to_tev))

    # Custom legend using "proxy artists" for full control
    legend_elements = [
        Line2D([0], [0], color='black', lw=3, linestyle=(0, (5, 0)), label='Observed'),
        Line2D([0], [0], color='black', lw=3, linestyle=(0, (5, 1)), label='Median expected'),
        Line2D([0], [0], color='black', lw=2, linestyle=(0, (3, 3)), label=r'$\pm 1\sigma$ Expected'),
        Line2D([0], [0], color='black', lw=2, linestyle=(0, (1, 5)), label=r'$\pm 2\sigma$ Expected')
    ]
    ax.legend(handles=legend_elements, loc='upper left', frameon=False, fontsize=29)
    ax.tick_params(axis='both', which='major', labelsize=26)

    ax.text(4900,1450,'\n'.join([r"$H_{D}$ mass = "+df['mhs'][0]+' GeV','Majorana DM',]), ha='right',va='top', fontsize=28)

    fig.tight_layout()
    fname = 'darkhiggs_'+df['mhs'][0]+'GeV_2d_limit.pdf'
    fig.savefig(fname)

In [ ]:
for idf in [mhs50_df, mhs70_df, mhs90_df, mhs110_df, mhs130_df, mhs150_df]:
    plot2d(idf)